In [1]:
import os
import numpy as np
import pandas as pd
import re
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'CS839_project'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['Project Ideas.gdoc', 'anmol_predictions', 'Dataset', 'entity_correction.ipynb', 'tables_reference_correction.ipynb']


In [9]:
import json

schema_path = os.path.join(GOOGLE_DRIVE_PATH,'Dataset/tables.json')
# data_path = os.path.join(GOOGLE_DRIVE_PATH,'Dataset/train_spider.json')

def get_entities(schema_path):
    with open(schema_path, 'r', encoding='utf8') as f:
        table_datas = json.load(f)

    output_tab = {}
    tables = {}
    tabel_name = set()
    for i in range(len(table_datas)):
        table = table_datas[i]
        temp = {}
        temp['col_map'] = table['column_names']
        temp['table_names'] = table['table_names']
        tmp_col = []
        for cc in [x[1] for x in table['column_names_original']]:
            if cc not in tmp_col:
                tmp_col.append(cc)
        table['col_set'] = tmp_col
        db_name = table['db_id']
        tabel_name.add(db_name)
        table['schema_content'] = [col[1] for col in table['column_names']]
        table['col_table'] = [col[0] for col in table['column_names']]
        output_tab[db_name] = temp
        tables[db_name] = table

    return tables

db_dict = get_entities(schema_path)

# Use 'col_set' from db_dict[<db_id>'] to match column names
# Use 'table_names' from db_dict[<db_id>] to match table names

In [ ]:
db_dict[db_id]

In [118]:
db_id = 'network_1'
pred = 'SELECT T1.name, count(*), sum(B.grade), avg(student_id), c.liked_id FROM Highschooler AS T join Friend as T2 join Likes as A'.lower()
# pred = 'SELECT count(number), sum(abc), T.ncv, A1.xyz, grade FROM High_School AS T1 left join aaaaa as T2 join'.lower()

In [140]:
def correct_table_reference(db_dict, db_id, pred):
  db = db_dict[db_id]
  dict_1 = {}

  for info in db['column_names_original']:
    key = info[1].lower()
    value = db['table_names_original'][info[0]]
    if key in dict_1:
      old_value = dict_1[key]
    else:
      old_value = []
    old_value.append(value.lower())
    dict_1[key] = old_value

  table_vars = re.findall(r'(\w+) as (\w+)', pred)
  # print(table_vars)

  dict_2 = {}
  for key, val in table_vars:
    dict_2[key] = val

  columns = re.findall(r'select (.*?) from', pred)
  if columns != []:
    columns = columns[0]
    columns = columns.split(',')

    agg_fn_lst = ['count','sum','avg','min','max']
    for agg_fn in agg_fn_lst:
      columns = [term.replace(agg_fn+'(','') for term in columns]
      columns = [term.replace(')','') for term in columns]

    columns = [term.strip() for term in columns]
    if '*' in columns:
      columns.remove('*')
    print(dict_1, dict_2)
    print(columns)
    dict_3 = {}
    for col in columns:
      if '.' in col:
        [table_var, col_name] = col.split('.')[-2:]
        if col_name in dict_1:
          for table_name in dict_1[col_name]:
            if table_name in dict_2:
              print(table_var, dict_2[table_name])
              pred = pred.replace(' '+table_var+'.', ' '+dict_2[table_name]+'.', 1)
              break

  return pred

In [141]:

db_id = 'student_transcripts_tracking'
pred = 'select avg(t1.t1.first_name) from students as'
print(db_id)
print(pred)
pred = correct_table_reference(db_dict, db_id, pred)
print(pred)

student_transcripts_tracking
select avg(t1.t1.first_name) from students as
{'*': ['transcript_contents'], 'address_id': ['addresses'], 'line_1': ['addresses'], 'line_2': ['addresses'], 'line_3': ['addresses'], 'city': ['addresses'], 'zip_postcode': ['addresses'], 'state_province_county': ['addresses'], 'country': ['addresses'], 'other_address_details': ['addresses'], 'course_id': ['courses', 'sections', 'student_enrolment_courses'], 'course_name': ['courses'], 'course_description': ['courses'], 'other_details': ['courses', 'departments', 'degree_programs', 'sections', 'semesters', 'student_enrolment', 'transcripts'], 'department_id': ['departments', 'degree_programs'], 'department_name': ['departments'], 'department_description': ['departments'], 'degree_program_id': ['degree_programs', 'student_enrolment'], 'degree_summary_name': ['degree_programs'], 'degree_summary_description': ['degree_programs'], 'section_id': ['sections'], 'section_name': ['sections'], 'section_description': ['se

In [ ]:
results = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH,'anmol_predictions/Results_ec.csv'))
row = results.iloc[0]
i = 0
preds_ec = []
for idx, row in results.iterrows():
  print(i)
  db_id = row['DB']
  gold = row['Gold']
  pred = row['Preds_ec'].lower()
  print(db_id)
  print(pred)
  # try:
  pred = correct_table_reference(db_dict, db_id, pred)
  # except:
  #   pass
  print(pred)
  preds_ec.append(pred)
  # if i==10:
  #   break
  i+=1

In [152]:
# results['Preds_ec_trb'] = preds_ec
# results.to_csv(os.path.join(GOOGLE_DRIVE_PATH,'anmol_predictions/results_improved.csv'), index=False)